<h1 style='font-size:55px'>
An Intro to Ciw
<img src='logo.png' align='left' height='125' width='125' style='float: left; margin-right: 40px; margin-top: 0px;'/>
</h1>
<font size='5'> Henry Wilde | 
<i class='fa fa-github' aria-hidden='false'></i>
<i class='fa fa-twitter' aria-hidden='false'></i> @daffidwilde | M/1.10B </font>
<hr>


# Key features:
---
- Discrete event simulation library built by Geraint Palmer
- Capability to handle complex queueing networks
- Advanced system qualities: patient baulking, classes, priorities, and server scheduling

Full tutorials can be found here: http://ciw.readthedocs.io/en/latest/index.html

# Installation
---
The quickest way is to use `pip`.

First, you'll need Python installed on your machine. 

It is highly recommended to use the Anaconda distribution for this. It can be downloaded here: https://www.anaconda.com/download/. With Anaconda, you have access to various interactive interfaces for running Python code (like this Jupyter Notebook).

So, to install Ciw.

1. Open up your command line
  - Mac/Linux use your **Terminal** app
  - Windows use the **Anaconda Prompt**
2. Type `pip install ciw` and press Return

# Basic usage
---
Suppose we wanted to model a toll point on a bridge.

- Here, **motorists arrive** into a single queue at a rate of **90 per minute** and there are **15 toll booths**
- On average, it takes **10 seconds** to serve each customer
- We assume that all arrivals and services are **exponentially distributed**

This system can be modelled using the M|M|10 queueing model.

## Creating a network, and running a simulation

- We must specify all arrival and service distributions, and the number of servers.

- In Ciw, we deal with nodes (in our case the 15 toll booths form one node with 15 servers). 

- When we run our simulations, it is good practice to 'seed' beforehand.

In [ ]:
import ciw


In [ ]:
# Create a network


In [ ]:
# Seed your work


In [ ]:
# Run a simulation of the network


## Looking at the system


In [ ]:
# Nodes


In [ ]:
# Individuals at a particular node


In [ ]:
# Average server utilisation


## Data records and collection

### On the individual level


In [ ]:
# Get an individual's data records


### System-wide records


In [ ]:
# Obtain data records for all individuals


### Manipulating our data


# Queueing networks
---

**Imagine a café that sells both hot and cold food. Customers arrive and can take a few routes:**

- Customers only wanting cold food must queue at the cold food counter, and then take their food to the till to pay.
- Customers only wanting hot food must queue at the hot food counter, and then take their food to the till to pay.
- Customers wanting both hot and cold food must first queue for cold food, then hot food, and then take both to the till and pay.

In this system there are **three nodes**: Cold food counter (Node 1), Hot food counter (Node 2), and the till (Node 3):

- Customers wanting **hot food only arrive** at a rate of **12 per hour.**
- Customers wanting **cold food arrive** at a rate of **18 per hour.**
- **30%** of all customer who buy **cold food also want to buy hot food.**
- On average it takes **1 minute** to be **served cold food**, **2 and a half minutes** to be **served hot food**, and **2 minutes to pay**.
- There is **1 server** at the **cold food** counter, **2 servers** at the **hot food** counter, and **2 servers** at the **till**.

This system can be drawn out as a diagram like so:

![](cafe-system.svg)

It follows that:

- An arrival rate of 18 per hour is equivalent to 0.2 per minute, and 12 per hour to 0.3 per minute.
- Average service times of 1, 2.5 and 2 minutes are equivalent to 1 per minute, 0.4 per minute and 0.5 per minute, respectively.

Using Ciw, we can construct this network like so:

In [ ]:
N = ciw.create_network(
    Arrival_distributions=[["Exponential", 0.3], ["Exponential", 0.2], "NoArrivals"],
    Service_distributions=[
        ["Exponential", 1.0],
        ["Exponential", 0.4],
        ["Exponential", 0.5],
    ],
    Transition_matrices=[[0.0, 0.3, 0.7], [0.0, 0.0, 1.0], [0.0, 0.0, 0.0]],
    Number_of_servers=[1, 2, 2],
)


Let us suppose we wish to simulate a 3 hour (180 minute) lunch shift. When the café opens, it is empty. Therefore, no warm-up time is needed in our simulation, but we will make use of 20 minutes of cool-down time.

Let us run 10 trials of this simulation and find the average number of customers to pass through the system.

In [ ]:
completed_customers = []
for trial in range(10):
    ciw.seed(trial)
    Q = ciw.Simulation(N)
    Q.simulate_until_max_time(200)
    recs = Q.get_all_records()
    num_completed = len([r for r in recs if r.node == 3 and r.arrival_date < 180])
    completed_customers.append(num_completed)


In [ ]:
sum(completed_customers) / len(completed_customers)


# Scheduling
---

Suppose we are modelling a reception desk for a brand new office building. There are various receptionists on duty throughout the day, and the desk is open constantly. The number of receptionists on duty follows a daily schedule like so:

| Time of day | 0:00 - 06:00 | 06:00 - 09:00 | 09:00 - 17:00 | 17:00 - 22:00 | 22:00 - 00:00 |
| ------------|-----|-----|------|-------|-------|
| Number of receptionists | 1 | 2 | 3 | 2 | 1 |

Every query to the desk takes 1 minute, and the arrival rate of people making a query at the desk is dependent on the time of day. We have that one person arrives every:
- 90 minutes from 22:00 to 06:00
- 15 minutes between 06:00 and 0:900
- 6 minutes between 09:00 and 17:00
- 30 minutes between 17:00 and 22:00

We can model this with Ciw as follows:

In [ ]:
def arrival_function(t):
    if t % 24 < 6:
        return 1.5
    if t % 24 < 9:
        return 0.25
    if t % 24 < 17:
        return 0.1
    if t % 24 < 22:
        return 0.5
    return 1.5


scheduling_network = ciw.create_network(
    Arrival_distributions=[["TimeDependent", arrival_function]],
    Service_distributions=[["Deterministic", 1]],
    Number_of_servers=[[[1, 6], [2, 9], [3, 17], [2, 22], [1, 24]]],
)


# Useful tools
---
## Pandas

Pandas is an intuitive tool for handling large amounts of data

In [ ]:
import pandas as pd


In [ ]:
ciw.seed(0)
Q = ciw.Simulation(N)
Q.simulate_until_max_time(200)

recs = Q.get_all_records()


In [ ]:
# Visualise data records as a dataframe


In [ ]:
# Manipulation of this data


## Matplotlib

An extensive ecosystem for plotting and visualising data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sbn

%matplotlib inline

sbn.set(style='darkgrid')

In [ ]:
fig = plt.figure(figsize=(12, 8))

plt.hist(df[df.node == 3].service_time, label="Node 3", alpha=0.6)
plt.hist(df[df.node == 2].service_time, label="Node 2", alpha=0.5)
plt.hist(df[df.node == 1].service_time, label="Node 1", alpha=0.5)

plt.legend(loc="best")
plt.title("Service time at each node")
plt.xlabel("Service time")


In [ ]:
node_3 = df[df.node == 3]

fig = plt.figure(figsize=(12, 8))
sbn.kdeplot(node_3.queue_size_at_arrival, node_3.queue_size_at_departure)
plt.title("Bivariate KDE plot for queue size at arrival and departure for Node 3")
